In [136]:
!pip install pyspark

In [137]:
from pyspark.sql import SparkSession

In [213]:
spark = SparkSession.builder.appName('rec').getOrCreate()

In [214]:
data = spark.read.csv('movielens_ratings.csv',inferSchema=True,header=True)

In [215]:
data.head()

Row(movieId=2, rating=3.0, userId=0)

In [216]:
data.describe().show()

+-------+------------------+------------------+------------------+
|summary|           movieId|            rating|            userId|
+-------+------------------+------------------+------------------+
|  count|              1501|              1501|              1501|
|   mean| 49.40572951365756|1.7741505662891406|14.383744170552964|
| stddev|28.937034065088994| 1.187276166124803| 8.591040424293272|
|    min|                 0|               1.0|                 0|
|    max|                99|               5.0|                29|
+-------+------------------+------------------+------------------+



In [217]:
(training, test) = data.randomSplit([0.8, 0.2])

In [219]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS

In [220]:
als = ALS(maxIter=5, regParam=0.01, userCol="userId", itemCol="movieId", ratingCol="rating")
model = als.fit(training)

In [221]:
predictions = model.transform(test)

In [222]:
predictions.show()

+-------+------+------+-----------+
|movieId|rating|userId| prediction|
+-------+------+------+-----------+
|      0|   3.0|    28| 0.38884866|
|      1|   1.0|    28|  -4.823266|
|      3|   2.0|    22|  2.3853295|
|      3|   1.0|    13|  1.2616079|
|      2|   3.0|     6|-0.41861135|
|      0|   1.0|     3| 0.60900205|
|      1|   1.0|    20|  1.2824459|
|      2|   2.0|    20| -1.3807023|
|      2|   1.0|    19|-0.00545913|
|      0|   1.0|     8|  1.0549874|
|      3|   2.0|     8|  3.8561404|
|      4|   1.0|    23|  1.0149773|
|      4|   1.0|     7|  1.2509109|
|      0|   3.0|    10|  0.9998375|
|      1|   3.0|    25|  3.0079117|
|      3|   1.0|    29|   0.209504|
|      4|   1.0|    29|  2.3794942|
|      1|   1.0|    14|  1.1450403|
|      4|   1.0|    14| 0.67601746|
|      2|   3.0|     0|  0.6522946|
+-------+------+------+-----------+
only showing top 20 rows



In [223]:
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",predictionCol="prediction")
rmse = evaluator.evaluate(predictions)
print("Root-mean-square error = " + str(rmse))

Root-mean-square error = 1.8470656792315587


In [224]:
single_user = test.filter(test['userId']==11).select(['movieId','userId'])

In [225]:
single_user.show()

+-------+------+
|movieId|userId|
+-------+------+
|      9|    11|
|     13|    11|
|     18|    11|
|     19|    11|
|     23|    11|
|     39|    11|
|     41|    11|
|     70|    11|
|     72|    11|
|     75|    11|
|     82|    11|
|     89|    11|
+-------+------+



In [226]:
reccomendations = model.transform(single_user)

In [227]:
reccomendations.orderBy('prediction',ascending=False).show()

+-------+------+----------+
|movieId|userId|prediction|
+-------+------+----------+
|     41|    11|   4.55385|
|     39|    11|  4.406943|
|      9|    11| 3.5625591|
|     89|    11| 3.4879022|
|     19|    11| 3.2384014|
|     23|    11| 3.1728396|
|     13|    11| 2.2196548|
|     75|    11| 1.3696997|
|     82|    11| 1.2324307|
|     70|    11| 0.6274237|
|     72|    11|0.29770637|
|     18|    11|0.13117556|
+-------+------+----------+

